# Auto Schema Writer

---

The purpose of this Notebook is to automate the tedious process of writing the code for the Entity Relationship Diagram and the PostgreSQL schema creation. This is all to fulfill the requirements of the Project 2 Crowdfunding_ETL assignment, as a more streamlined way of doing so all in one file would be with SQL Alchemy as opposed to creating text output to be copy and pasted or written to a file!

In this case I chose the latter just to show how easy it is to write something that auto generates a SQL script to write as necessary.  
Some of this was a little hardcoded but that was due to the lack of this being a sufficiently complex repository/project to warrant that.

In [13]:
from pathlib import Path
import pandas as pd

## f(x): Schema Writer
---
A generator function that is used to read the new CSV files and return schemas for each one.  
Can be used to create both the ERD code for QuickDraw or for the PostgreSQL schema creation itself.  

The results returned are simply to help ensure integrity in the results transferred to pgAdmin 4, as opposed to less calculated methods.  
After copying and pasting only slight modifications and Primary/Foreign Key constraints are required.

In [14]:
def schema_info(
        path_suffix: str
        ,start: int = None
        ,stop: int = None
        ,root: str = '../../Resources/Output'
    ):
    type_map = {
        'int64': 'INT'
        ,'float64': 'DEC'
        ,'object': 'CHAR'
    }
    path = f'{root}/{path_suffix}.csv'
    df = pd.read_csv(path)

    columns = [*df.columns]
    for col in columns[start:stop]:
        counts = df[col].astype(str).str.len()
        min = counts.min()
        max = counts.max()
        _type = type_map[df[col].dtype.name]
        if _type == 'CHAR':
            if min != max:
                _type = 'VARCHAR'
            result = f'{col} {_type}({max})'
        else:
            result = f'{col} {_type}'
        yield result

### ERD Code for Quick Draw

In [15]:
csv_files = ['campaign', 'contacts', 'category', 'subcategory']

for csv in csv_files:
    print(
        f'{csv}\n' +
        '-'
    )
    for col in schema_info(csv):
        print(col)
    print()

campaign
-
cf_id INT
contact_id INT
company_name VARCHAR(33)
description VARCHAR(53)
goal DEC
pledged DEC
outcome VARCHAR(10)
backers_count INT
country CHAR(2)
currency CHAR(3)
launch_date CHAR(10)
end_date CHAR(10)
category_id CHAR(4)
subcategory_id VARCHAR(8)

contacts
-
contact_id INT
first_name VARCHAR(12)
last_name VARCHAR(13)
email VARCHAR(42)

category
-
category_id CHAR(4)
category VARCHAR(12)

subcategory
-
subcategory_id VARCHAR(8)
subcategory VARCHAR(17)



### PostgreSQL Schema Code

In [16]:
csv_files = ['contacts', 'category', 'subcategory', 'campaign']

db_schema_lines = ['-- Schema Creation and Validation\n']

db_schema_lines += \
    [f'DROP TABLE IF EXISTS {csv} CASCADE;\n' for csv in csv_files]
db_schema_lines += '\n'

for csv in csv_files:
    db_schema_lines.append(
        f'CREATE TABLE {csv} (\n' +
        f'    {next(schema_info(csv))}\n'
    )
    for col in schema_info(csv, 1):
        constr = ''
        cond1 = 'end_date' in col
        cond2 = 'description' in col
        cond3 = 'id' not in col
        if cond1 or cond2:
            pass
        elif cond3: 
            constr = 'NOT NULL'
        db_schema_lines.append(f'    ,{col} {constr}\n')
    db_schema_lines.append(
        ');\n\n' +
        f'SELECT * FROM {csv};\n\n'
    )

db_schema_lines.append('\n-- CSV Import Validation\n')
db_schema_lines += [f'SELECT * FROM {csv};\n' for csv in csv_files]
db_schema_lines += '\n'

In [17]:
for line in db_schema_lines[:15]:
    print(line)

-- Schema Creation and Validation

DROP TABLE IF EXISTS contacts CASCADE;

DROP TABLE IF EXISTS category CASCADE;

DROP TABLE IF EXISTS subcategory CASCADE;

DROP TABLE IF EXISTS campaign CASCADE;



CREATE TABLE contacts (
    contact_id INT

    ,first_name VARCHAR(12) NOT NULL

    ,last_name VARCHAR(13) NOT NULL

    ,email VARCHAR(42) NOT NULL

);

SELECT * FROM contacts;


CREATE TABLE category (
    category_id CHAR(4)

    ,category VARCHAR(12) NOT NULL

);

SELECT * FROM category;


CREATE TABLE subcategory (
    subcategory_id VARCHAR(8)



In [18]:
with open('RAW_autoSchema.sql', 'w') as sql:
    sql.writelines(db_schema_lines)

---

---

## f(x): CSV Import Writer
When used, generates statements in SQL that can be copied and pasted into a `.sql` file in the `crowdfunding_db` after cloning repository.  
Must be run after after `schema_info()` has been utilized to create a schema and added to `crowdfunding_db` as well.

In [19]:
def import_info(sortBy: list, parent_path: str = '../../Resources/Output'):
    dir_path = sorted(
        [*Path(parent_path).resolve().iterdir()]
        ,key = lambda path: sortBy.index(path.stem)
    )
    for count, csv in enumerate(dir_path, 1):
        result = (
            f"-- ({count})\n" +
            "COPY\n" +
            f"    {csv.stem}\n" +
            "FROM\n" +
            f"    '{csv}'\n" +
            "DELIMITER ',' CSV HEADER;\n\n"
        )
        yield result

### PostgreSQL Import Code

In [20]:
csv_files = ['contacts', 'category', 'subcategory', 'campaign']
db_import_lines = ['-- Local CSV POSIX Path from Drive\n']
db_import_lines.append('-- Written from schema_writer.ipynb safely\n\n')
db_import_lines += [sql_stmt for sql_stmt in import_info(sortBy = csv_files)]

In [21]:
for line in db_import_lines:
    print(line)

-- Local CSV POSIX Path from Drive

-- Written from schema_writer.ipynb safely


-- (1)
COPY
    contacts
FROM
    '/Users/neelagarwal/Projects/DataClassRepos/Crowdfunding_ETL/Resources/Output/contacts.csv'
DELIMITER ',' CSV HEADER;


-- (2)
COPY
    category
FROM
    '/Users/neelagarwal/Projects/DataClassRepos/Crowdfunding_ETL/Resources/Output/category.csv'
DELIMITER ',' CSV HEADER;


-- (3)
COPY
    subcategory
FROM
    '/Users/neelagarwal/Projects/DataClassRepos/Crowdfunding_ETL/Resources/Output/subcategory.csv'
DELIMITER ',' CSV HEADER;


-- (4)
COPY
    campaign
FROM
    '/Users/neelagarwal/Projects/DataClassRepos/Crowdfunding_ETL/Resources/Output/campaign.csv'
DELIMITER ',' CSV HEADER;




In [22]:
with open('crowdfunding_db_import.sql', 'w') as sql:
    sql.writelines(db_import_lines)